In [6]:
import pandas as pd
from obspy import read
import matplotlib.pyplot as plt
import os
from sklearn.preprocessing import MinMaxScaler
from scipy.signal import find_peaks
from scipy import stats
import numpy as np



In [3]:
'''
# Make Test catalog: Lunar data

import os
import pandas as pd

main_path = 'data/lunar/test/'

list_dfs = []

for root, dirs, files in os.walk(main_path):
    for file in files:
        if file.endswith('.csv'):
            trackfile = os.path.join(root, file)
            df = pd.read_csv(trackfile)
            df['filename'] = file
            list_dfs.append(df)

#Concat all Dataframes
df_concat = pd.concat(list_dfs, ignore_index=True)

#Save
df_concat.to_csv('data/lunar/test/data/lunar_catalogs.csv', index=False)

print('Your new CSV with all Test data CSV is ready!')


'''

"\n# Make Test catalog\n\nimport os\nimport pandas as pd\n\nmain_path = 'data/lunar/test/'\n\nlist_dfs = []\n\nfor root, dirs, files in os.walk(main_path):\n    for file in files:\n        if file.endswith('.csv'):\n            trackfile = os.path.join(root, file)\n            df = pd.read_csv(trackfile)\n            df['filename'] = file\n            list_dfs.append(df)\n\n#Concat all Dataframes\ndf_concat = pd.concat(list_dfs, ignore_index=True)\n\n#Save\ndf_concat.to_csv('data/lunar/test/data/lunar_catalogs.csv', index=False)\n\nprint('Your new CSV with all Test data CSV is ready!')\n\n\n"

In [4]:
data = pd.read_csv('data/lunar/test/data/lunar_catalogs.csv') #generated concat data
data.head(5)

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s),filename
0,1970-05-23T00:00:00.147000,0.000000,-1.276555e-15,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv
1,1970-05-23T00:00:00.297943,0.150943,-1.553765e-15,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv
2,1970-05-23T00:00:00.448887,0.301887,-1.647706e-15,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv
3,1970-05-23T00:00:00.599830,0.452830,-1.560061e-15,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv
4,1970-05-23T00:00:00.750774,0.603774,-1.358306e-15,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv


Data Processing

In [7]:
scaler = MinMaxScaler()
data['velocity(m/s)'] = scaler.fit_transform(data[['velocity(m/s)']])
window_size = 10  # Definir o tamanho da janela de suavização
data['velocity_smooth'] = data['velocity(m/s)'].rolling(window=window_size).mean()

peaks, _ = find_peaks(data['velocity_smooth'], height=0.01)  
data['is_sismo'] = 0
data.loc[peaks, 'is_sismo'] = 1  

data['log_velocity'] = np.log1p(data['velocity(m/s)'].abs()) 

data['z_score'] = np.abs(stats.zscore(data['velocity(m/s)']))
sismos = data[data['z_score'] > 0.1]  #Considering z-score = 0.1
sismos

,time_abs(%Y-%m-%dT%H:%M:%S.%f),time_rel(sec),velocity(m/s),filename,velocity_smooth,velocity_diff,is_sismo,log_velocity,z_score
2190,1970-05-23T00:05:30.713038,330.566038,0.460971,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv,0.461266,-0.000056,0,0.379101,0.112465
2226,1970-05-23T00:05:36.147000,336.000000,0.460954,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv,0.461273,-0.000089,0,0.379090,0.118709
2270,1970-05-23T00:05:42.788509,342.641509,0.461549,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv,0.461309,0.000098,0,0.379497,0.102375
2397,1970-05-23T00:06:01.958321,361.811321,0.460925,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv,0.461290,-0.000198,0,0.379070,0.129654
2398,1970-05-23T00:06:02.109264,361.962264,0.460896,xa.s12.00.mhz.1970-05-23HR00_evid00027.csv,0.461243,-0.000029,0,0.379050,0.140421
...,...,...,...,...,...,...,...,...,...
40360529,1974-11-21T11:50:02.129434,42601.509434,0.460308,xa.s15.00.mhz.1974-11-21HR00_evid00166.csv,0.461417,-0.001190,0,0.378647,0.359177
40360530,1974-11-21T11:50:02.280377,42601.660377,0.459631,xa.s15.00.mhz.1974-11-21HR00_evid00166.csv,0.461179,-0.000676,0,0.378184,0.610663
40360531,1974-11-21T11:50:02.431321,42601.811321,0.459809,xa.s15.00.mhz.1974-11-21HR00_evid00166.csv,0.461032,0.000177,0,0.378305,0.544759
40360532,1974-11-21T11:50:02.582264,42601.962264,0.460589,xa.s15.00.mhz.1974-11-21HR00_evid00166.csv,0.461038,0.000780,0,0.378840,0.254702


In [8]:
sismos.info()

<class 'pandas.core.frame.DataFrame'>
Index: 22617891 entries, 2190 to 40360534
Data columns (total 9 columns):
 #   Column                          Dtype  
---  ------                          -----  
 0   time_abs(%Y-%m-%dT%H:%M:%S.%f)  object 
 1   time_rel(sec)                   float64
 2   velocity(m/s)                   float64
 3   filename                        object 
 4   velocity_smooth                 float64
 5   velocity_diff                   float64
 6   is_sismo                        int64  
 7   log_velocity                    float64
 8   z_score                         float64
dtypes: float64(6), int64(1), object(2)
memory usage: 1.7+ GB


In [11]:
# Data manipulation
sismos.loc[:, 'time_rel(sec)'] = data['time_rel(sec)'].astype(float).round(0).astype(int)
sismos.loc[:, 'velocity(m/s)'] = data['velocity(m/s)'].astype(float).round(0).astype(int)

In [13]:
np.float = np
np.int = np

In [15]:
import numpy as np
import pandas as pd
from skmultiflow.data import DataStream
from skmultiflow.meta import AdaptiveRandomForestClassifier
from skmultiflow.evaluation import EvaluatePrequential
from scipy.signal import find_peaks
from sklearn.preprocessing import MinMaxScaler
from scipy import stats


X = sismos[['time_rel(sec)', 'velocity(m/s)']].values
y = sismos['is_sismo'].values  

stream = DataStream(X, y)

model = AdaptiveRandomForestClassifier()

evaluator = EvaluatePrequential(
    pretrain_size=100,  
    max_time=1000, 
    max_samples=len(X),  
    batch_size=1,  
    n_wait=100,  
    show_plot=False,  
    metrics=['accuracy', 'precision', 'recall', 'f1']  
)

evaluator.evaluate(stream=stream, model=[model], model_names=['ARF'])

evaluation_summary = evaluator.get_mean_measurements()
print(evaluation_summary)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 100 sample(s).
Evaluating...

Time limit reached (1000.00s). Evaluation stopped.
Processed samples: 15376
Mean performance:
ARF - Accuracy     : 0.8926
ARF - Precision: 0.0091
ARF - Recall: 0.0014
ARF - F1 score: 0.0024
